In [844]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import tensorflow as tf
import numpy as np

tf.__version__



'1.4.0'

In [845]:
# x->x를 예측

PAD = 0
START = 1
EOS = 2

num_class = 7
hidden_unit = 7
num_layer = 3
batch_size = 3

encoder_input = [
    [5,6,3,4],
    [6,5,3,0],
    [5,6,0,0]
]

decoder_input = [
    [START,5,6,3,4],
    [START,6,5,3,0],
    [START,5,6,0,0]
]

target = [
    [5,6,3,4,EOS],
    [6,5,3,EOS,0],
    [5,6,EOS,0,0]
]

cell_type = "LSTM"

In [846]:
encoder_input

[[5, 6, 3, 4], [6, 5, 3, 0], [5, 6, 0, 0]]

In [847]:
decoder_input

[[1, 5, 6, 3, 4], [1, 6, 5, 3, 0], [1, 5, 6, 0, 0]]

In [848]:
target

[[5, 6, 3, 4, 2], [6, 5, 3, 2, 0], [5, 6, 2, 0, 0]]

In [834]:
def model_fn(mode, features, labels,params):
    
    def get_single_cell(cell_type):
        if cell_type == "GRU":
            single_cell = tf.contrib.rnn.GRUCell(
                hidden_unit)
        elif cell_type == "LSTM":
            single_cell = tf.contrib.rnn.BasicLSTMCell(
                hidden_unit,
                forget_bias=1.0)
        return single_cell

    if type(features) == dict:
        print(features)
        encoder_input = features["encoder_input"]
        decoder_input = features["decoder_input"]

    encoder_input_one_hot = tf.one_hot(encoder_input,depth=num_class,dtype=tf.float32)
    decoder_input_one_hot = tf.one_hot(decoder_input,depth=num_class,dtype=tf.float32)
    
    with tf.variable_scope('encoder'):
        
        fw_cells = [] 
        bw_cells = []
        
        for i in range(0,num_layer):
            fw_cells.append(get_single_cell(cell_type))
            bw_cells.append(get_single_cell(cell_type))
            
        (outputs, output_state_fw, output_state_bw) = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                        cells_bw=bw_cells,
                                                        cells_fw=fw_cells,
                                                        inputs=encoder_input_one_hot,
                                                        dtype=tf.float32,
                                                        time_major=False)
        
        if cell_type == "LSTM":
            #lstm과 나머지 cell의 state의 형태가 다르다. LSTM은 tuple형태
            encoder_final_state_c = tf.concat((output_state_fw[-1].c,output_state_bw[-1].c),1)
            encoder_final_state_h = tf.concat((output_state_fw[-1].h,output_state_bw[-1].h),1)
            encoder_final_state = tf.contrib.rnn.LSTMStateTuple(c = encoder_final_state_c, h = encoder_final_state_h)
        else:
            encoder_final_state = tf.concat((output_state_fw[-1],output_state_bw[-1]),1)
    
    with tf.variable_scope('decoder'):
        decoder_cell = tf.contrib.rnn.LSTMCell(num_units=hidden_unit*2,state_is_tuple=True)
        decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
            decoder_cell, decoder_input_one_hot,
            initial_state=encoder_final_state,
            dtype=tf.float32)
        
    decoder_logits = tf.contrib.layers.fully_connected(decoder_outputs,num_class,activation_fn=None)
    decoder_prediction = tf.argmax(decoder_logits, 2)

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=decoder_prediction)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(labels,depth=num_class,dtype=tf.float32),
            logits=decoder_logits,
        )
        
        loss = tf.reduce_mean(stepwise_cross_entropy,name="loss")
            
        train_op = tf.contrib.layers.optimize_loss(
            loss, tf.train.get_global_step(),
            optimizer='Adam',
            summaries=['loss', 'learning_rate'],
            learning_rate=0.001,
            name="train_op")
        
        predictions = decoder_prediction
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            predictions=predictions,
            train_op=train_op)
   

In [835]:
run_config = tf.contrib.learn.RunConfig(model_dir="./model_dir",save_summary_steps=500)

In [837]:
estimator = tf.estimator.Estimator(config=run_config,model_fn=model_fn,model_dir="./model_dir",params=None,)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1253c0630>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model_dir'}


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1253c0630>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model_dir'}


In [838]:
my_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"encoder_input":np.array(encoder_input,dtype=np.int32),"decoder_input":np.array(decoder_input,dtype=np.int32)},
    y=np.array(target,dtype=np.int32),
    batch_size=batch_size,
    shuffle=False,
    num_epochs=None)

In [840]:

estimator.train(input_fn = my_input_fn,hooks=None,steps=1000,max_steps=None,saving_listeners=None)

{'decoder_input': <tf.Tensor 'fifo_queue_DequeueMany:1' shape=(3, 5) dtype=int32>, 'encoder_input': <tf.Tensor 'fifo_queue_DequeueMany:2' shape=(3, 4) dtype=int32>}
INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into ./model_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into ./model_dir/model.ckpt.


INFO:tensorflow:loss = 1.97522, step = 1


...

INFO:tensorflow:loss = 0.124501, step = 901 (0.795 sec)


INFO:tensorflow:Saving checkpoints for 1000 into ./model_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into ./model_dir/model.ckpt.


INFO:tensorflow:Loss for final step: 0.117022.


INFO:tensorflow:Loss for final step: 0.117022.


In [841]:
my_predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"encoder_input":np.array(encoder_input,dtype=np.int32),"decoder_input":np.array(decoder_input,dtype=np.int32)},
    y=np.array(decoder_input,dtype=np.int32),
    batch_size=batch_size,
    shuffle=False,
    num_epochs=1)

In [842]:
result = estimator.predict(input_fn=my_predict_input_fn)

In [843]:
for label,pred in zip(target,result):
    print(label,pred)

{'decoder_input': <tf.Tensor 'fifo_queue_DequeueUpTo:1' shape=(?, 5) dtype=int32>, 'encoder_input': <tf.Tensor 'fifo_queue_DequeueUpTo:2' shape=(?, 4) dtype=int32>}
INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-1000


INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-1000


[5, 6, 3, 4, 2] [5 6 3 4 2]
[6, 5, 3, 2, 0] [5 5 3 2 0]
[5, 6, 2, 0, 0] [5 6 2 0 0]
